In [1]:
!pip install Faker

from faker import Faker
fake = Faker(locale='en_US')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
fake = Faker(locale='en_US')
record= fake.simple_profile()
            # fake_records= fake.simple_profile()
record['birthdate'] =  str(record['birthdate'])
print(record)

{'username': 'whitecatherine', 'name': 'Tim Lee', 'sex': 'M', 'address': '29908 Gina Views\nShannonfort, KS 41002', 'mail': 'bergsamantha@hotmail.com', 'birthdate': '2004-11-04'}


In [3]:
# source: https://github.com/karmus89/blockchain-a-z

import datetime
import hashlib
import json
import sys


class Blockchain:

    def __init__(self):
        
        self.chain = []
        self.create_block(proof=1, previous_hash='0', record = {'username': '', 'name': '', 'sex': '', 'address': '', 'mail': '', 'birthdate': ''})

    def create_block(self, proof, previous_hash,record):
        "Create a block."

        block = {
            'index': len(self.chain),
            'timestamp': str(datetime.datetime.now()),
            'proof': proof,
            'previous_hash': previous_hash,
            'record': record
        }
        self.chain.append(block)
        
        return block

    def get_previous_block(self):
        
        return self.chain[-1]

    def hash_proof(self, previous_proof, next_proof):
        "Calculate the SHA256-hash"
        
        return (hashlib
                .sha256(str(next_proof**2 - previous_proof**2).encode())
                .hexdigest())

    def hash_block(self, block):
        "Calculate the SHA256-hash for a block."

        encoded_block = json.dumps(block, sort_keys=True).encode()
        
        return hashlib.sha256(encoded_block).hexdigest()

    def proof_of_work(self, previous_proof):
        "Calculate a new proof related to the previous block."

        next_proof = 1
        check_proof = False

        while check_proof is False:

            if self.hash_proof(previous_proof, next_proof)[:4] == '0000':
                check_proof = True

            else:
                next_proof += 1

        return next_proof

    def is_chain_valid(self, chain):
        "Validate that block and proof hashes are correct across the chain."

        for i in range(len(chain)):

            if i == 0:
                continue

            if chain[i]['previous_hash'] != self.hash_block(chain[i-1]):
                return False

            previous_proof = chain[i-1]['proof']
            next_proof = chain[i]['proof']

            if self.hash_proof(previous_proof, next_proof)[:4] != '0000':
                
                return False

        return True

In [4]:
import threading
import requests

from flask import Flask, request, jsonify
from werkzeug.serving import run_simple
from faker import Faker



class BlockchainApp:

    def __init__(self, host='localhost', port=5000, chain=Blockchain):

        self.host = host
        self.port = port
        self.chain = chain()

        self.host_url = f'http://{self.host}:{self.port}'
        
        self.app = Flask(__name__)
        self.add_api_endpoints()
        
        self.thread = threading.Thread(
            target=run_simple,
            kwargs={
                'hostname': self.host,
                'port': self.port,
                'application': self.app}
        )

    def __enter__(self):

        self.start()

        return self

    def __exit__(self, *args):

        self.stop()
        
    def add_api_endpoints(self):
        "Add API endpoints to the Flask WebApp."
        
        self.app.add_url_rule(
            rule='/blocks',
            view_func=self.api_blocks,
            methods=['GET', 'POST']
        )
        self.app.add_url_rule(
            rule='/blocks/validate',
            view_func=self.api_validate,
        )
        self.app.add_url_rule(
            rule='/shutdown',
            view_func=self.api_shutdown,
        )

    def api_blocks(self):
        "Either retrieve the node's current chain or post a new block to the chain."

        if request.method == 'POST':

            prev_block = self.chain.get_previous_block()
            prev_hash = self.chain.hash_block(prev_block)
            prev_proof = prev_block['proof']

            proof = self.chain.proof_of_work(prev_proof)

            

            
            fake = Faker(locale='en_US')
            record= fake.simple_profile()
            # fake_records= fake.simple_profile()
            record['birthdate'] =  str(record['birthdate'])

            block = self.chain.create_block(proof, prev_hash,record)

            response = {'message': 'Congratulations, you just mined a Block!',
                        'block': block}

            return (jsonify(response), 200)

        if request.method == 'GET':

            response = {'chain': self.chain.chain,
                        'length': len(self.chain.chain)}

            return (jsonify(response), 200)

    def api_validate(self):
        "Validate the chain"

        if self.chain.is_chain_valid(self.chain.chain):

            response = {'message': 'Chain is valid.',
                        'valid': True}
            return (jsonify(response), 200)

        else:

            response = {'message': 'Chain is not valid!',
                        'valid': False}
            return (jsonify(response), 500)

    def api_shutdown(self):
        "Shutdown the Flask WebApp"

        # depricated
        # request.environ.get('werkzeug.server.shutdown')()

        return jsonify({'message': 'Shutting down'}), 200

    def start(self):
        "Start the Flask-based Blockchain WebApp."

        self.thread.start()

    def stop(self):
        "Stop the Flask-based Blockchain WebApp."

        if self.thread.is_alive():

            return requests.get(f'{self.host_url}/shutdown')

In [5]:
import time

with BlockchainApp() as blockchain_app:

    # wait for server thread ready
    time.sleep(5)

    for _ in range(19):

        response = requests.post(f'{blockchain_app.host_url}/blocks')
        print("Proof={}".format(response.json()['block']['proof']))

    response = requests.get(f'{blockchain_app.host_url}/blocks')
    print("Chain={}".format(json.dumps(response.json(), indent=2)))

    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))

    for i in enumerate(blockchain_app.chain.chain):
      print(i)

    # original = blockchain_app.chain.chain[1]['proof']
    # blockchain_app.chain.chain[1]['proof'] = 12345
    # print('Proofs: Original={}, Forged={}'.format(original,blockchain_app.chain.chain[1]['proof']))
    
    # response = requests.get(f'{blockchain_app.host_url}/blocks')
    # print(json.dumps(response.json(), indent=2))
    
    response = requests.get(f'{blockchain_app.host_url}/blocks/validate')
    print(json.dumps(response.json(), indent=2))
    
    # proof_1 = blockchain_app.chain.chain[1]['proof']
    # proof_2 = blockchain_app.chain.chain[2]['proof']

    # forged_hash = blockchain_app.chain.hash_proof(proof_1, proof_2)

    # print("Hashes: \n\tOriginal =\t{}\n\tForged =\t{}".format(blockchain_app.chain.chain[2]['previous_hash'],
    #                                                           forged_hash))


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:16] "POST /blocks HTTP/1.1" 200 -


Proof=533


INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:17] "POST /blocks HTTP/1.1" 200 -


Proof=45293


INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:17] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:17] "POST /blocks HTTP/1.1" 200 -


...

Proof=144125
Proof=2492
Proof=22592


INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "POST /blocks HTTP/1.1" 200 -


Proof=107780
Proof=47346


INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "POST /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "GET /blocks HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "GET /blocks/validate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [14/Apr/2023 17:53:20] "GET /shutdown HTTP/1.1" 200 -


Proof=46891
Chain={
  "chain": [
    {
      "index": 0,
      "previous_hash": "0",
      "proof": 1,
      "record": {
        "address": "",
        "birthdate": "",
        "mail": "",
        "name": "",
        "sex": "",
        "username": ""
      },
      "timestamp": "2023-04-14 17:53:11.711888"
    },
    {
      "index": 1,
      "previous_hash": "e502e4b4dbd0fb3aab2d8fe9f1075bd23a93ade06c2c6f3347ecd200994d16d0",
      "proof": 533,
      "record": {
        "address": "59039 Foster Parks Apt. 795\nKristinchester, NH 10362",
        "birthdate": "1992-06-18",
        "mail": "michael25@gmail.com",
        "name": "Mary Morgan",
        "sex": "F",
        "username": "susangarcia"
      },
      "timestamp": "2023-04-14 17:53:16.844121"
    },
    {
      "index": 2,
      "previous_hash": "35988c21d8fe2cb302f0b08c50c4c6191f9ba957c634a79be874c06ad590461c",
      "proof": 45293,
      "record": {
        "address": "674 Patel Fort Apt. 084\nBradleymouth, MS 31746",
        

In [6]:
!apt install net-tools
!netstat -tulnp | grep :5000

Reading package lists... Done
Building dependency tree       
Reading state information... Done
net-tools is already the newest version (1.60+git20180626.aebd88e-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
tcp        0      0 127.0.0.1:5000          0.0.0.0:*               LISTEN      2733/python3        


In [7]:
!kill 2253

/bin/bash: line 0: kill: (2253) - No such process
